In [1]:
import pandas as pd
import re

In [6]:
# Do this for 
# cohort = 'System'
# cohort = 'LiberalArts' and 
# cohort = 'Peers'
cohort = 'Peers'
df = pd.read_csv(f'TeacherStudentIPEDS{cohort}.csv')
df

,UnitID,Institution Name,Sector of institution (HD2023),Instructional staff on 9 10 11 or 12 month contract-total (SAL2023_IS All full-time instructional staff total),Salary outlays - total (SAL2023_IS All full-time instructional staff total),Grand total (EF2023 All students Undergraduate total),Grand total (EF2023 All students Graduate and First professional),Instructional staff on 9 10 11 or 12 month contract-total (SAL2022_IS_RV All full-time instructional staff total),Salary outlays - total (SAL2022_IS_RV All full-time instructional staff total),Grand total (EF2022_RV All students Undergraduate total),...,Grand total (EF2014_RV All students Graduate and First professional),Instructional staff on 9 10 11 or 12 month contract-total (SAL2013_IS_RV All full-time instructional staff total),Salary outlays - total (SAL2013_IS_RV All full-time instructional staff total),Grand total (EF2013_RV All students Undergraduate total),Grand total (EF2013_RV All students Graduate and First professional),Instructional staff on 9 10 11 or 12 month contract-total (SAL2012_IS_RV All full-time instructional staff total),Salary outlays - total (SAL2012_IS_RV All full-time instructional staff total),Grand total (EF2012_RV All students Undergraduate total),Grand total (EF2012_RV All students Graduate and First professional),Unnamed: 51
0,231712,Christopher Newport University,1,277,26444761,4407,96.0,280,25164046,4449,...,125.0,268,18584837,5094,138.0,258,17030306,5046,140.0,NaN
1,218070,Furman University,2,270,25278288,2327,178.0,265,24041527,2283,...,163.0,244,19696015,2797,156.0,246,19677953,2753,162.0,NaN
2,167288,Massachusetts College of Liberal Arts,1,79,6652694,836,97.0,79,6636132,838,...,203.0,87,6352184,1538,178.0,85,6141771,1600,199.0,NaN
3,262129,New College of Florida,1,100,8570474,710,22.0,97,7802818,669,...,NaN,75,5415181,793,NaN,72,4994586,832,NaN,NaN
4,186201,Ramapo College of New Jersey,1,190,22263129,5013,508.0,192,22309454,4932,...,293.0,216,20564495,5614,238.0,218,20110811,5624,193.0,NaN
5,163912,St. Mary's College of Maryland,1,134,11808807,1577,10.0,122,10394318,1497,...,33.0,153,10729238,1819,39.0,150,10524490,1901,32.0,NaN
6,196219,SUNY at Purchase College,1,182,14513197,3165,64.0,188,14440842,3107,...,101.0,169,11999726,4239,114.0,160,11609581,4128,112.0,NaN
7,196167,SUNY College at Geneseo,1,214,18216433,3868,60.0,211,17189272,4110,...,110.0,253,18157169,5504,140.0,255,18171929,5388,169.0,NaN
8,206589,The College of Wooster,2,168,13029794,1876,NaN,177,13697286,1967,...,NaN,176,12683602,2102,NaN,170,11908315,2080,NaN,NaN
9,232681,University of Mary Washington,1,224,19277330,3611,197.0,236,19341510,3493,...,368.0,245,17360895,4383,448.0,244,16764226,4515,578.0,NaN


In [7]:
# Columns to preserve
id_cols = [
    'UnitID', 'Institution Name', 
    # 'Institution name alias (HD2023)', 
    'Sector of institution (HD2023)'
]

df_long = df.melt(id_vars=id_cols, var_name='column', value_name='cnt')
df_long = df_long.dropna(subset=['cnt'])
df_long = df_long[df_long['cnt'].astype(str).str.strip() != '']

def extract_year_variable(col):
    if match := re.search(r'(SAL|EF)(\d{4})', col):
        kind, year = match.groups()
        if "Instructional staff" in col:
            return int(year), "TeacherCnt"
        elif "Undergraduate total" in col:
            return int(year), "UndergraduateCnt"
        elif "Graduate and First professional" in col:
            return int(year), "GraduateCnt"
        elif "Salary" in col:
            return int(year), "SalaryOutlay"
    return None, None

df_long[['Year', 'Variable']] = df_long['column'].apply(lambda x: pd.Series(extract_year_variable(x)))
df_long = df_long.dropna(subset=['Year', 'Variable'])


# Pivot to tidy format
df_tidy = df_long.pivot_table(
    index=[
        'UnitID', 'Institution Name', 
        # 'Institution name alias (HD2023)', 
        'Sector of institution (HD2023)', 
        'Year'
    ],
    columns='Variable',
    values='cnt',
    aggfunc='first'
).reset_index()

# Rename columns
df_tidy.rename(columns={
    'UnitID': 'SchoolID',
    'Institution Name': 'SchoolName',
    # 'Institution name alias (HD2023)': 'SchoolAlias',
    'Sector of institution (HD2023)': 'Sector'
}, inplace=True)

# Convert count columns to numeric
for col in ['UndergraduateCnt', 'GraduateCnt', 'SalaryOutlay', 'TeacherCnt']:
    df_tidy[col] = pd.to_numeric(df_tidy[col], errors='coerce').fillna(0).apply(int)

df_tidy['Cohort'] = cohort
df_tidy

Variable,SchoolID,SchoolName,Sector,Year,GraduateCnt,SalaryOutlay,TeacherCnt,UndergraduateCnt,Cohort
0,163912,St. Mary's College of Maryland,1,2012,32,10524490,150,1901,Peers
1,163912,St. Mary's College of Maryland,1,2013,39,10729238,153,1819,Peers
2,163912,St. Mary's College of Maryland,1,2014,33,10732381,147,1688,Peers
3,163912,St. Mary's College of Maryland,1,2015,27,10491780,146,1708,Peers
4,163912,St. Mary's College of Maryland,1,2016,31,10523756,143,1598,Peers
...,...,...,...,...,...,...,...,...,...
151,262129,New College of Florida,1,2019,24,8147741,105,702,Peers
152,262129,New College of Florida,1,2020,29,8368586,108,646,Peers
153,262129,New College of Florida,1,2021,27,7956314,103,632,Peers
154,262129,New College of Florida,1,2022,20,7802818,97,669,Peers


In [8]:
df_tidy.to_csv(f'{cohort}StudentsAndTeachers.csv', index=False)

In [9]:
dfPeers = pd.read_csv('PeersStudentsAndTeachers.csv')
dfSystem = pd.read_csv('SystemStudentsAndTeachers.csv')
# dfLiberalArts = pd.read_csv('LiberalArtsStudentsAndTeachers.csv')
dfMerged = pd.concat([dfSystem, dfPeers])
dfMerged

,SchoolID,SchoolName,Sector,Year,GraduateCnt,SalaryOutlay,TeacherCnt,UndergraduateCnt,Cohort
0,197869,Appalachian State University,1,2012,1877,62327615,901,15712,System
1,197869,Appalachian State University,1,2013,1813,62484822,901,16025,System
2,197869,Appalachian State University,1,2014,1771,63872066,920,16255,System
3,197869,Appalachian State University,1,2015,1642,65648607,930,16290,System
4,197869,Appalachian State University,1,2016,1700,71149902,951,16595,System
...,...,...,...,...,...,...,...,...,...
151,262129,New College of Florida,1,2019,24,8147741,105,702,Peers
152,262129,New College of Florida,1,2020,29,8368586,108,646,Peers
153,262129,New College of Florida,1,2021,27,7956314,103,632,Peers
154,262129,New College of Florida,1,2022,20,7802818,97,669,Peers


In [10]:
dfMerged.to_csv('StudentTeacherData.csv', index=False)

In [ ]:
# Columns to preserve
id_cols = [
    'UnitID', 'Institution Name', 
    # 'Institution name alias (HD2023)', 
    'Sector of institution (HD2023)'
]

df_long = df.melt(id_vars=id_cols, var_name='column', value_name='cnt')
df_long = df_long.dropna(subset=['cnt'])
df_long = df_long[df_long['cnt'].astype(str).str.strip() != '']

def extract_year_variable(col):
    if match := re.search(r'(SAL|EF)(\d{4})', col):
        kind, year = match.groups()
        if "Instructional staff" in col:
            return int(year), "TeacherCnt"
        elif "Undergraduate total" in col:
            return int(year), "UndergraduateCnt"
        elif "Graduate and First professional" in col:
            return int(year), "GraduateCnt"
    return None, None

df_long[['Year', 'Variable']] = df_long['column'].apply(lambda x: pd.Series(extract_year_variable(x)))
df_long = df_long.dropna(subset=['Year', 'Variable'])

# Pivot to tidy format
df_tidy = df_long.pivot_table(
    index=[
        'UnitID', 'Institution Name', 
        # 'Institution name alias (HD2023)', 
        'Sector of institution (HD2023)', 
        'Year'
    ],
    columns='Variable',
    values='cnt',
    aggfunc='first'
).reset_index()

# Rename columns
df_tidy.rename(columns={
    'UnitID': 'SchoolID',
    'Institution Name': 'SchoolName',
    # 'Institution name alias (HD2023)': 'SchoolAlias',
    'Sector of institution (HD2023)': 'Sector'
}, inplace=True)

# Convert count columns to numeric
for col in ['TeacherCnt', 'UndergraduateCnt', 'GraduateCnt']:
    df_tidy[col] = pd.to_numeric(df_tidy[col], errors='coerce').fillna(0)

# Compute ratios
df_tidy['StudentTeacherRatioOverall'] = (
    (df_tidy['UndergraduateCnt'] + df_tidy['GraduateCnt']) / df_tidy['TeacherCnt']
).replace([float('inf'), -float('inf')], pd.NA)

df_tidy['StudentTeacherRatioUndergraduate'] = (
    df_tidy['UndergraduateCnt'] / df_tidy['TeacherCnt']
).replace([float('inf'), -float('inf')], pd.NA)

# Compute RelativeChange
df_tidy.sort_values(by=['SchoolID', 'Year'], inplace=True)
ref_pop = df_tidy[df_tidy['Year'] == 2012].set_index('SchoolID')['UndergraduateCnt'] + \
          df_tidy[df_tidy['Year'] == 2012].set_index('SchoolID')['GraduateCnt']
df_tidy['ReferencePopulation'] = df_tidy['SchoolID'].map(ref_pop)
df_tidy['CurrentPopulation'] = df_tidy['UndergraduateCnt'] + df_tidy['GraduateCnt']
df_tidy['RelativeChange'] = df_tidy['CurrentPopulation'] / df_tidy['ReferencePopulation']
df_tidy.loc[df_tidy['Year'] == 2012, 'RelativeChange'] = 1.0
df_tidy.drop(columns=['ReferencePopulation', 'CurrentPopulation'], inplace=True)
df_tidy['Cohort'] = cohort

df_final = df_tidy[['SchoolID', 'SchoolName', #'SchoolAlias', 
    'Cohort', 'Sector', 'Year', 
    'TeacherCnt', 'UndergraduateCnt', 'GraduateCnt', 
    'StudentTeacherRatioOverall', 'StudentTeacherRatioUndergraduate', 'RelativeChange']]
df_final

In [ ]:
df_final.to_csv(f'{cohort}Data.csv', index=False)

In [ ]:
dfSystem = pd.read_csv('SystemData.csv')
dfLiberalArts = pd.read_csv('LiberalArtsData.csv')
dfMerged = pd.concat([dfSystem, dfLiberalArts])
dfMerged

In [ ]:
dfMerged.to_csv('MergedData.csv', index=False)

In [ ]:
df = pd.read_csv("TeacherStudentIPEDSLiberalArts.csv")

# Keep ID and Name columns
id_vars = [
    'UnitID', 'Institution Name', 
    # 'Institution name alias (HD2023)', 
    # 'State abbreviation (HD2023)', 
    'Sector of institution (HD2023)'
]

# All other columns are value columns
value_vars = [col for col in df.columns if col not in id_vars]

# Melt the dataframe to long format
df_long = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='column', value_name='cnt')

# Drop rows with missing or empty counts
df_long = df_long.dropna(subset=['cnt'])
df_long = df_long[df_long['cnt'].astype(str).str.strip() != '']

# Function to extract year and variable
def extract_year_variable(col_name):
    year_match = re.search(r'(19|20)\d{2}', col_name)
    year = int(year_match.group(0)) if year_match else None

    if 'Undergraduate' in col_name:
        variable = 'undergraduate_student_cnt'
    elif 'Graduate and First professional' in col_name:
        variable = 'graduate_student_cnt'
    elif ('instruction' in col_name):
        variable = 'teacher_cnt'
    else:
        variable = None  # We'll drop these

    return pd.Series({'year': year, 'variable': variable})

# Apply the extractor function
df_long = df_long.join(df_long['column'].apply(extract_year_variable))

# Drop rows without a recognized variable or year
df_clean = df_long.dropna(subset=['year', 'variable'])

# Rename for final output
df_clean = df_clean.rename(columns={
    'UnitID': 'SchoolID',
    'Institution Name': 'SchoolName',
    'Sector of institution (HD2023)': 'Sector'
})

# Keep only relevant columns
df_final = df_clean[['SchoolID', 'SchoolName', 'Sector', 'year', 'variable', 'cnt']]

df_final2 = df_final[(df_final.SchoolID != 433174) & (df_final.SchoolID != 199184)]
df_final2['cnt'] = df_final2['cnt'].apply(int)
df_final2['year'] = df_final2['year'].apply(int)

# df_final2.to_csv('system_schools.csv', index=False)


In [ ]:
# Pivot to get counts for ratio computation
df_wide = df_final2.pivot_table(
    index=['SchoolID', 'SchoolName', 'year'],
    columns='variable',
    values='cnt',
    aggfunc='first'
).reset_index()

# Fill missing values with 0
df_wide = df_wide.fillna(0)

# Compute ratios
df_wide['student_teacher_ratio'] = (
    (df_wide['undergraduate_student_cnt'] + df_wide['graduate_student_cnt']) / df_wide['teacher_cnt']
).replace([float('inf'), -float('inf')], None)

df_wide['ug_student_teacher_ratio'] = (
    df_wide['undergraduate_student_cnt'] / df_wide['teacher_cnt']
).replace([float('inf'), -float('inf')], None)

# Convert ratio columns to tidy format
ratio_long = df_wide.melt(
    id_vars=['SchoolID', 'SchoolName', 'year'],
    value_vars=['student_teacher_ratio', 'ug_student_teacher_ratio'],
    var_name='variable',
    value_name='cnt'
)

# Ensure cnt is numeric
ratio_long['cnt'] = ratio_long['cnt'].astype(float)

# Combine with original tidy data
df_tidy_with_ratios = pd.concat([df_final2, ratio_long], ignore_index=True)

# Save
df_tidy_with_ratios.to_csv("liberal_arts_schools.csv", index=False)


In [ ]:

# Columns to preserve directly
id_cols = [
    'UnitID', 
    'Institution Name', 
    'Institution name alias (HD2023)', 
    'Sector of institution (HD2023)'
]

# Melt the rest of the data into long format
df_long = df.melt(id_vars=id_cols, var_name='column', value_name='cnt')

# Drop empty or null rows
df_long = df_long.dropna(subset=['cnt'])
df_long = df_long[df_long['cnt'].astype(str).str.strip() != '']

# Extract year and variable type from the column name
def extract_year_variable(col):
    if match := re.search(r'(SAL|EF)(\d{4})', col):
        kind, year = match.groups()
        if "Instructional staff" in col:
            return int(year), "TeacherCnt"
        elif "Undergraduate total" in col:
            return int(year), "UndergraduateCnt"
        elif "Graduate and First professional" in col:
            return int(year), "GraduateCnt"
    return None, None

df_long[['Year', 'Variable']] = df_long['column'].apply(lambda x: pd.Series(extract_year_variable(x)))
df_long = df_long.dropna(subset=['Year', 'Variable'])

# Pivot to wide format
df_tidy = df_long.pivot_table(
    index=[
        'UnitID', 'Institution Name', 
        'Institution name alias (HD2023)', 
        'Sector of institution (HD2023)', 
        'Year'
    ],
    columns='Variable',
    values='cnt',
    aggfunc='first'
).reset_index()

# Rename and convert types
df_tidy.rename(columns={
    'UnitID': 'SchoolID',
    'Institution Name': 'SchoolName',
    'Institution name alias (HD2023)': 'SchoolAlias',
    'Sector of institution (HD2023)': 'Sector'
}, inplace=True)

# Convert values to numeric for ratio calculations
for col in ['TeacherCnt', 'UndergraduateCnt', 'GraduateCnt']:
    df_tidy[col] = pd.to_numeric(df_tidy[col], errors='coerce')

# Compute ratios
df_tidy['StudentTeacherRatioOverall'] = (df_tidy['UndergraduateCnt'] + df_tidy['GraduateCnt']) / df_tidy['TeacherCnt']
df_tidy['StudentTeacherRatioUndergraduate'] = df_tidy['UndergraduateCnt'] / df_tidy['TeacherCnt']

# Replace NaNs in count columns with 0
df_tidy[['GraduateCnt', 'UndergraduateCnt', 'TeacherCnt']] = df_tidy[
    ['GraduateCnt', 'UndergraduateCnt', 'TeacherCnt']
].fillna(0)

# Recompute ratios now that nulls are gone
df_tidy['StudentTeacherRatioOverall'] = (
    (df_tidy['UndergraduateCnt'] + df_tidy['GraduateCnt']) / df_tidy['TeacherCnt']
).replace([float('inf'), -float('inf')], pd.NA)

df_tidy['StudentTeacherRatioUndergraduate'] = (
    df_tidy['UndergraduateCnt'] / df_tidy['TeacherCnt']
).replace([float('inf'), -float('inf')], pd.NA)

# Show the result
# import ace_tools as tools; tools.display_dataframe_to_user(name="Tidy IPEDS Data", dataframe=df_tidy)

# df_tidy = df_tidy.dropna()
df_tidy['Cohort'] = cohort
df_tidy['Year'] = df_tidy.Year.apply(int)
df_tidy['GraduateCnt'] = df_tidy.GraduateCnt.apply(int)
df_tidy['TeacherCnt'] = df_tidy.TeacherCnt.apply(int)
df_tidy['UndergraduateCnt'] = df_tidy.UndergraduateCnt.apply(int)

df_final = df_tidy[['SchoolID', 'SchoolName', 'SchoolAlias', 'Cohort', 'Sector', 'Year', 'TeacherCnt', 'UndergraduateCnt', 'GraduateCnt', 'StudentTeacherRatioOverall', 'StudentTeacherRatioUndergraduate']]
df_final